In [19]:
import pandas as pd

games = pd.read_csv("games_cleaned.csv",index_col="appid")

In [20]:
games

,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,header_image,...,median_playtime_forever,median_playtime_2weeks,discount,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent,tags_text
appid,,,,,,,,,,,,,,,,,,,,,
730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,https://shared.akamai.steamstatic.com/store_it...,...,5174,350,0,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473,FPS Shooter Multiplayer Competitive Action Tea...
578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,https://shared.akamai.steamstatic.com/store_it...,...,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720,Survival Shooter Battle Royale Multiplayer FPS...
570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,https://shared.akamai.steamstatic.com/store_it...,...,898,892,0,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366,Free to Play MOBA Multiplayer Strategy e-sport...
271590,Grand Theft Auto V Legacy,2015-04-13,17,0.00,0,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,https://shared.akamai.steamstatic.com/store_it...,...,7101,74,0,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517,Open World Action Multiplayer Crime Automobile...
359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,3.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,https://shared.akamai.steamstatic.com/store_it...,...,2434,306,80,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608,FPS PvP e-sports Multiplayer Tactical Shooter ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2115110,Outrun Them,2024-02-29,0,3.99,0,The name of the game is literally the game. Ou...,The name of the game is literally the game. Ou...,It's all in the name.. outrun them.,NaN,https://shared.akamai.steamstatic.com/store_it...,...,0,0,0,0,"{'2D Platformer': 91, 'Runner': 85, 'Platforme...",-1,-1,-1,-1,2D Platformer Runner Platformer 2D Top-Down Ca...
1174200,Lands of Pharaoh: Episode 1,2020-01-07,0,10.00,0,Join Our Discord Channel About the Game Briefl...,Briefly: Lands of Pharaoh is a third person vi...,Lands of Pharaoh is a multi-camera-angle based...,NaN,https://shared.akamai.steamstatic.com/store_it...,...,0,0,0,0,"{'Action': 43, 'Adventure': 41, 'Indie': 41, '...",-1,-1,-1,-1,Action Adventure Indie Strategy
1160190,Player One,2019-10-10,0,1.99,0,Player One is a new 3D game where you take con...,Player One is a new 3D game where you take con...,"3D Game, Thirdperson Platformer",NaN,https://shared.akamai.steamstatic.com/store_it...,...,0,0,0,0,"{'Indie': 31, 'Early Access': 21}",-1,-1,-1,-1,Indie Early Access


In [21]:
games["tags_text"]

appid
730        FPS Shooter Multiplayer Competitive Action Tea...
578080     Survival Shooter Battle Royale Multiplayer FPS...
570        Free to Play MOBA Multiplayer Strategy e-sport...
271590     Open World Action Multiplayer Crime Automobile...
359550     FPS PvP e-sports Multiplayer Tactical Shooter ...
                                 ...                        
2115110    2D Platformer Runner Platformer 2D Top-Down Ca...
1174200                      Action Adventure Indie Strategy
1160190                                   Indie Early Access
3380340    RPG Action-Adventure Strategy Simulation Survi...
763730                                          Action Indie
Name: tags_text, Length: 89618, dtype: object

In [22]:
games["tags_text"].to_csv("tags_text.txt", index=True, header=False)

In [23]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3070


In [24]:
from sentence_transformers import SentenceTransformer
import numpy as np
import os

emb_file = "embeddings.npy"
appids_file = "appids.npy"

games = pd.read_csv("games_cleaned.csv", index_col="appid")

if os.path.exists(emb_file) and os.path.exists(appids_file):
    print("Embeddings già presenti, li carico...")
    embeddings_final = np.load(emb_file)
    appids = np.load(appids_file)
else:
    print("Embeddings non trovati, li calcolo...")
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device='cuda')
    desc_texts = games["short_description"].fillna("").tolist()
    tags_texts = games["tags_text"].fillna("").tolist()
    embeddings_desc = model.encode(desc_texts, convert_to_numpy=True, show_progress_bar=True)
    embeddings_tags = model.encode(tags_texts, convert_to_numpy=True, show_progress_bar=True)

    embeddings_combined = 0.7 * embeddings_tags + 0.3 * embeddings_desc
    num_reviews = games["num_reviews_total"].fillna(0).to_numpy()
    num_reviews_norm = (num_reviews - num_reviews.min()) / (num_reviews.max() - num_reviews.min())
    num_reviews_norm = num_reviews_norm.reshape(-1, 1)

    embeddings_final = np.hstack([embeddings_combined, 0.20 * num_reviews_norm])

    appids = games.index.to_numpy()

    # salva gli embeddings e appid nella cartella del progetto github
    np.save(emb_file, embeddings_final)
    np.save(appids_file, appids)
    print("Embeddings calcolati e salvati!")

Embeddings non trovati, li calcolo...


Batches: 100%|██████████| 2801/2801 [00:18<00:00, 148.37it/s]


Embeddings calcolati e salvati!


In [25]:

embeddings_df = pd.DataFrame(embeddings_final, index=appids)
embeddings_df.columns = [f"embedding_{i}" for i in range(embeddings_final.shape[1])]

print(embeddings_df)

         embedding_0  embedding_1  embedding_2  embedding_3  embedding_4  \
730        -0.007127     0.016883    -0.023919    -0.039693     0.003072   
578080      0.007103    -0.001423    -0.014612    -0.057371     0.021316   
570        -0.049573     0.028742    -0.009435    -0.078919    -0.014215   
271590     -0.014976    -0.009490    -0.050807    -0.055450     0.043457   
359550     -0.015434     0.047119    -0.011894    -0.066275    -0.031378   
...              ...          ...          ...          ...          ...   
2115110     0.016484     0.005756    -0.021061    -0.062670     0.033130   
1174200     0.046598    -0.005016     0.002827    -0.020156    -0.024836   
1160190     0.006604    -0.071893     0.001208    -0.038882    -0.004639   
3380340    -0.031040     0.020907    -0.043146    -0.065624    -0.013790   
763730      0.012012    -0.064026     0.020826     0.008518     0.010527   

         embedding_5  embedding_6  embedding_7  embedding_8  embedding_9  ...  \
730   